# Computer Vision

## Assignment - Gaussian Mixture Model (GMM)

### Install

In [ ]:
%pip install comet_ml --quiet

### Comet Config

In [ ]:
from comet_ml import Experiment

experiment = Experiment(project_name = "cv-gmm")

### Imports

In [ ]:
import time
from os import path

import matplotlib.pyplot as plt

import numpy as np
import scipy

from cv_gmm_deep.datasets import load_data
from cv_gmm_deep.common import split_data, preprocess_data
from cv_gmm_deep.gmm import GMM

print(f"numpy=={np.__version__}")
print(f"scipy=={scipy.__version__}")

### Constants

In [ ]:
assets_dir = './assets'
data_dir = './data'

default_input_size = [768, 1024]
default_input_dir = path.join(
    assets_dir,
    f'puzzle_corners_{default_input_size[1]}x{default_input_size[0]}')

time_now = int(time.time())

default_cp_path = path.join(data_dir, str(time_now), 'cp.ckpt')
default_model_path = path.join(data_dir, str(time_now), 'model.h5')

### Params

In [ ]:
params = {
    'batch_size': 2,
    'epochs': 2,
    'features': ['rgb', 'DoG'],
    'input_size': default_input_size,
}

experiment.log_parameters(params)

### Setup

In [ ]:
def setup_args():
    return {
        'input_dir': default_input_dir,
        'model_path': default_model_path,
        'update_model': None
    }

### Train

In [ ]:
args = setup_args()

batch_size = params['batch_size']
epochs = params['epochs']
features = params['features']
input_size = params['input_size']

input_dir = args['input_dir']
model_path = args['model_path']
update_model = args['update_model']

height, width = input_size

In [ ]:
raw_x, raw_y = load_data(input_dir, height, width)

In [ ]:
data_x, data_y = preprocess_data(raw_x, raw_y, features)

In [ ]:
(train_x, train_y), (val_x, val_y), (test_x, test_y) =\
    split_data(data_x, data_y, ratios=[0.7, .15, .15])
    
experiment.log_dataset_hash(train_x)

In [ ]:
if update_model and model_path:
    model = GMM.load_model(model_path)
else:
    model = GMM(K=2, I=data_x.shape[-1])
    
assert model is not None, "Could not load model"

model.compile()

model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(val_x, val_y))

output, _ = model(train_x[:2])
plt.imshow(output[0], cmap="gray")

if model_path is not None:
    model.save(model_path)

### Comet Commit

In [ ]:
experiment.end()